# Neural style transfer

**Define object : Conserving the content of the original iamge while adopting the style of the reference image. **

Below, mathmatically define the Object 

$$
Loss(image \ to \ transfer) 
= distance(style(reference_image) - style(generated_image)) + \\
distance(content(original_image) - content(generated_image))
$$

* Distance 
  -  Norm kind fucntion to minimize 

* Content loss - Consistency between Original and Generated one 

 - would be captured in upper layers 
 - $L2$ norm between the activations of an upper layer in a pretrained convnet, compute over the target image, and the activations of the same layer computedd over the generated image. 
 - Use only a layer 

* Style

  - use Multiple layer 
  - Use $Gram \ matrix$ of a layer's activation : which is Inner product of the feature maps of a given layer. -> it represents a map of correlations between the layer's feature. These feature corelations capture the statistics of the patterns of a particular spatial scale, which empirically correspond to the appearance of the textures found at this scale. 
  - aims to preserve similar internal correlations within the activations of different layers, across the style-reference image and the generated image. 
  - It gurantees that the textures found at different spatial scales look similar across the style-reference image and the generated one.

### What I am doing 

- Preserve content by maintaing similar high-level layer activations between the target content image and the generated image. The convnet should see both the target iamge, and generated image as containing the same things.

- Preserve style by maintaining similar correlations within actvations for both low level layers and high-level layers. 
> Feature correlations capture textures : generated iamge and the syle reference image should share the same textures at different spatial scales. 

### Implementing 

* Pretrained model - VGG19

  1. Setting up a network that computes VGG19 layer activations for the style-feference image, the target image, and the generated image at the same time. 

  2.  Use the layer activations computed over these three iamges to define the loss function described earlier, which you'll minimize in order to achieve style transfer. 
  
  3. Setting up gradient-descent process to minimize this loss function.
  
  
- We use images which have 400px height fixxed (widely different sizes make style transger more difficult)

In [2]:
from keras.preprocessing.image import load_img, img_to_array

target_image_path = ''
style_reference_iamge_path = ''

width, height = load_img(target_image_path).size

img_height = 400 
img_width = int(width * img_height / height)

NameError: name 'load_image' is not defined

In [1]:
def preprocess_image(iamge_path):
    # this line to be refactored 
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img 

def deprocess_image(x):
    
    #### where these numbers came from
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    x = x[:, :, ::-1] # BGR to RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return x 

In [4]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_iamge = K.constant(preprocess_image(style_reference_iamge_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, 
                             style_reference_iamge,
                             combination_image], axis=0)


model = vgg19.VGG19(input_tensor=input_tensor,
                   weight='imagenet', include_top=False)
print("model loaded sucessfully")

NameError: name 'image_path' is not defined

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3 
    size = img_height * img_width
    
    return K.sum(K.square(S - C)) / (4. * (channel**2) * (size**2))

In [ ]:
def total_variation_loss(x) :
    a = 